In [ ]:
import pandas as pd
import json
import altair as alt
import json

# open five review json files at ./datasets/udi_reviews/reviews_[1-5].json
# and combine them into one pandas dataframe
df = pd.DataFrame()
for i in range(1, 6):
    with open(f'./datasets/udi_reviews/reviews_{i}.json', 'r') as f:
        data = json.load(f)
        single_df = pd.json_normalize(data)
        if i == 1:
            df = single_df
        else:
            df = pd.concat([df, single_df], ignore_index=True)

print(df.shape)
print(df['original_id'].nunique())
print(df.columns)

(357, 23)
274
Index(['id', 'data_id', 'original_id', 'combined_id', 'template_id',
       'expanded_id', 'paraphrased_id', 'query_template', 'constraints',
       'spec_template', 'query_type', 'creation_method', 'query_base', 'spec',
       'solution', 'dataset_schema', 'query', 'expertise', 'formality',
       'reviewer', 'review_status', 'review_comments', 'review_categories'],
      dtype='object')


In [345]:
reviewer_counts = df.groupby('reviewer').size().reset_index(name='counts')
# Create a bar chart for reviewer counts using Altair
chart = alt.Chart(reviewer_counts).mark_bar().encode(
    x=alt.X('reviewer:N', title='Reviewer', sort='-y'),
    y=alt.Y('counts:Q', title='Counts'),
    tooltip=['reviewer', 'counts']
).properties(
    title='Reviewer Counts'
).configure_axis(
    labelAngle=45
)

chart.display()

alt.Chart(...)

In [346]:
unique_template_ids = df['template_id'].nunique()
print(f"Number of unique template_id values: {unique_template_ids}")

Number of unique template_id values: 62


In [347]:
# Every template is seen at least once.
template_counts = df.groupby('template_id').size().reset_index(name='counts')
chart = alt.Chart(template_counts).mark_bar().encode(
    x=alt.X('template_id:O', title='Template'),
    y=alt.Y('counts:Q', title='Counts'),
    tooltip=['template_id', 'counts']
).properties(
    title='Template ID Counts'
).configure_axis(
    labelAngle=0
)

chart.display()

alt.Chart(...)

In [359]:
# overall, how "good" are the reviews?
scale = {"domain": ["good", "improve", "bad"], "range": ["#43A047", "#FFB300", "#E53935"]}
sort_key = {"good": 0, "improve": 1, "bad": 2}
color_legend = alt.Legend(
                        labelExpr="""
                        {
                            0: 'Good',
                            1: 'Improve',
                            2: 'Bad'
                        }[datum.label]
                        """
                    )

# review_status
reviewer_status = df.groupby('review_status').size().reset_index(name='counts')
#  convert counts into percentage
reviewer_status['percent'] = (reviewer_status['counts'] / reviewer_status['counts'].sum()) * 100
reviewer_status['sort_key'] = reviewer_status['review_status'].map(sort_key)
reviewer_status = reviewer_status.sort_values(by='sort_key')
print(reviewer_status.head())

chart = alt.Chart(reviewer_status).mark_bar().encode(
    x=alt.X('percent:Q', title='Percent'),
    color=alt.Color('sort_key:O', title='Percent', legend=color_legend).scale(domain=[0,1,2], range=scale['range']),
    tooltip=['review_status', 'percent']
).properties(
    title='Overall Scores'
).configure_axis(
    labelAngle=0
)

chart.save('./out/figures/review_overall.png')
chart.display()

  review_status  counts    percent  sort_key
1          good     271  75.910364         0
2       improve      59  16.526611         1
0           bad      27   7.563025         2


alt.Chart(...)

In [349]:
# overall, how "good" are the reviews?
reviewer_status = df.groupby(['reviewer','review_status']).size().reset_index(name='counts')
reviewer_status['reviewer_index'] = reviewer_status['reviewer'].astype('category').cat.codes + 1

reviewer_status['sort_key'] = reviewer_status['review_status'].map(sort_key)
reviewer_status = reviewer_status.sort_values(by='sort_key')

chart = alt.Chart(reviewer_status).mark_bar().encode(
    x=alt.X('counts:Q', title='Count').stack("normalize"),
    y=alt.X('reviewer_index:O', title='Reviewer'),
    color=alt.Color('sort_key:N', title='Percent', legend=color_legend).scale(domain=[0,1,2], range=scale['range']),
    tooltip=['review_status', 'counts']
).properties(
    title='Reviewer Scores'
).configure_axis(
    labelAngle=0
)

chart.save('./out/figures/review_score_dist.png')
chart.display()

alt.Chart(...)

In [358]:
# How similar are the reviews for the same exact question?
same_template = df.groupby('original_id')
# filter to reviews that have more than one review
same_template = same_template.filter(lambda x: len(x) > 4)
print(same_template.shape)
# for each question, get the reviewer_status distribution
same_template = same_template.groupby(['original_id', 'review_status']).size().reset_index(name='counts')
template_counts = same_template.groupby('original_id')['counts'].sum().reset_index(name='total_counts')

same_template = same_template.merge(template_counts, on='original_id')

#  convert counts into percentage
same_template['percent'] = (same_template['counts'] / same_template['total_counts']) * 100

same_template['sort_key'] = same_template['review_status'].map(sort_key)
same_template['sort_key_2'] = same_template.apply(lambda row: row['percent'] - (row['sort_key'] * 1000), axis=1)

same_template = same_template.sort_values(by=['sort_key_2'], ascending=False)

chart = alt.Chart(same_template).mark_bar().encode(
    x=alt.X('counts:Q', title='Count').stack("normalize"),
    y=alt.X('original_id:N', title='Question ID', sort=alt.SortField('sort_key_2', order='descending')),
    color=alt.Color('sort_key:N', title='Percent', legend=color_legend).scale(domain=[0,1,2], range=scale['range']),
    tooltip=['review_status', 'counts', 'sort_key', 'sort_key_2']
).properties(
    title='Reviewers Give Different Scores for the Same Question'
).configure_axis(
    labelAngle=0
)

chart.save('./out/figures/review_consistency.png')
chart.display()

(100, 23)


alt.Chart(...)

In [351]:
# number of answers for each formality / expertise score
paraphrased = df.groupby(['expertise', 'formality']).size().reset_index(name='counts')
chart = alt.Chart(paraphrased).mark_rect().encode(
    x=alt.X('expertise:O'),
    y=alt.Y('formality:O'),
    color=alt.Color('counts:Q', scale=alt.Scale(scheme='viridis'), title='Counts'),
    tooltip=['expertise', 'formality', 'counts']
).properties(
    title='Formality and Expertise Counts'
)

chart.display()

alt.Chart(...)

In [352]:
# is the proportion of 'good' reviewer_status the same for each formality / expertise score?
paraphrase_score = df.groupby(['expertise', 'formality']).size().reset_index(name='counts')
paraphrased_good = df[df['review_status'] == 'good'].groupby(['expertise', 'formality']).size().reset_index(name='counts')
print(paraphrased_good.shape)
print(paraphrase_score.shape)
paraphrased_good['percent'] = (paraphrased_good['counts'] / paraphrase_score['counts']) * 100
chart = alt.Chart(paraphrased_good).mark_rect().encode(
    x=alt.X('expertise:O'),
    y=alt.Y('formality:O'),
    color=alt.Color('percent:Q', scale=alt.Scale(scheme='viridis'), title='Percent Good'),
    tooltip=['expertise', 'formality', 'percent', 'counts']
).properties(
    title='Formality and Expertise Counts'
)

# This could be explored further, e.g. % bad, group by just formality or just expertise.
chart.display()

(25, 3)
(25, 3)


alt.Chart(...)

In [353]:
# get just the 'bad' reviews
bad_reviews = df[df['review_status'] == 'bad'].copy()
# expand the review_categories column into separate rows
bad_reviews['review_categories'] = bad_reviews['review_categories'].apply(json.loads)
bad_reviews = bad_reviews.explode('review_categories')

bad_reviews = bad_reviews.groupby('review_categories').size().reset_index(name='counts')
# convert counts into percentage
bad_reviews['percent'] = (bad_reviews['counts'] / bad_reviews['counts'].sum()) * 100

chart = alt.Chart(bad_reviews).mark_bar().encode(
    x=alt.X('percent:Q', title='Percent'),
    color=alt.Y('review_categories:N', title='Percent'),
    tooltip=['review_categories', 'percent']
).properties(
    title='Review Categories'
).configure_axis(
    labelAngle=0
)
chart.save('./out/figures/review_categories.png')
chart.display()



alt.Chart(...)

In [354]:
reviews_with_comments = df[df['review_comments'].notnull()]
reviews_with_comments = df[df['review_comments'] != ''].copy()
reviews_with_comments['comment_length'] = reviews_with_comments['review_comments'].apply(len)
chart = alt.Chart(reviews_with_comments).mark_bar().encode(
    x=alt.X('comment_length:Q', bin=alt.Bin(maxbins=20), title='Comment Length (characters)'),
    y=alt.Y('count()', title='Frequency'),
    tooltip=['comment_length', 'count()']
).properties(
    title='Review Comment Length'
)

chart.display()

alt.Chart(...)

In [355]:
reviews_with_comments = df[df['review_comments'].notnull()]
reviews_with_comments = df[df['review_comments'] != ''].copy()
reviews_with_comments = reviews_with_comments[['review_status','review_comments', 'review_categories', 'query', 'query_base']]
reviews_with_comments.to_json('./datasets/reviews_with_comments.json', orient='records', lines=True)
# reviewing some of the comments a few themes pop out:

# the large number of null values was noticed
# paraphrasing to 'weird' outputs "Would you please furnish the distribution..."
# paraphrasing conflated technical terms, especially to blood group.
# minor vis rendering things
# change vis encoding (e.g. swap x and y axis on scatterplot)
